In [4]:
import pandas as pd
tot = pd.read_csv("tot.csv")
tot
# tot_1 = tot.iloc[:,1:]

,연도,배출량,인구수,GRDP(단위 : 백만원),1인당 GRDP(단위 : 천원),관광객수
0,2014,976.2,621550,15147843,25968,12273917
1,2015,1162.3,641335,16946709,28280,13664395
2,2016,1305.3,661190,18719373,30284,15852980
3,2017,1312.1,678772,19981072,31470,14753236
4,2018,1311.5,692032,19910936,30506,14313961


In [7]:
tot_1.corr()

,배출량,인구수,GRDP(단위 : 백만원),1인당 GRDP(단위 : 천원),관광객수
배출량,1.000000,0.911172,0.964806,0.978952,0.898285
인구수,0.911172,1.000000,0.976099,0.916648,0.665335
GRDP(단위 : 백만원),0.964806,0.976099,1.000000,0.981353,0.778669
1인당 GRDP(단위 : 천원),0.978952,0.916648,0.981353,1.000000,0.852108
관광객수,0.898285,0.665335,0.778669,0.852108,1.000000


In [5]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins
print(folium.__version__)

0.10.1


In [6]:
state_geo = 'jeju.json'
state_geo
'지역', '세대수', '사업체수', '카드매출액', 'wifi사용자수', '클린하우스개수',

'jeju.json'

In [25]:
coding: 'UTF8'

In [11]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins

state_geo = 'jeju.geojson'
state_unemployment = '4.csv'
state_data = pd.read_csv(state_unemployment)
state_data.columns = ['군집', 'CODE']
state_data['CODE'] = state_data.CODE.map(lambda x : str(x).zfill(7))
state_data.head(1)

,군집,CODE
0,1,3901054


In [12]:
jeju_map = folium.Map(location=[33.363600, 126.528222], zoom_start = 10)
jeju_map.choropleth(geo_data=state_geo, data=state_data, 
               columns=['군집', 'CODE'],key_on = 'feature.properties.adm_cd',
               fill_color='PuRd', fill_opacity=0.01)
jeju_map

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [16]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [17]:
client_id = "j2nijytx8x"
client_pw = "D12z0TZrLISsTlZMNaDejzgrvYZnbn4rhdViIoKz"

In [18]:
api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

data = pd.read_excel('cleanhouse.xlsx', usecols='A, B, C',names=['읍면동', '도움센터명칭','주소'])

# 위도경도 찾기
geo_coordi = []
for add in data['주소']:
    add_urlenc = parse.quote(add)
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID',client_id)
    request.add_header('X-NCP-APIGW-API-KEY',client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'addresses' in response_body:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
            else:
                print("'result' not exist!")
                latitude = None
                longitude  = None
        else:
            print('Response error code : %d' %rescode)
            latitude = None
            longitude = None
            
    geo_coordi.append([latitude, longitude])

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!


In [19]:
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"읍면동":data["읍면동"].values,
                              "도움센터명칭":data["도움센터명칭"].values,
                              "도로명":data['주소'].values,
                              "위도": np_geo_coordi[:,0],
                              "경도": np_geo_coordi[:,1]})

In [20]:
writer = pd.ExcelWriter('clean_h.xlsx')
pd_geo_coordi.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [21]:
import folium

for clean, la, ln in zip(data["도움센터명칭"], np_geo_coordi[:,0], np_geo_coordi[:,1]):
    m = folium.Marker([la,ln], popup=clean)
    m.add_to(jeju_map)
jeju_map

In [22]:
for clean, la, ln in zip(data["도움센터명칭"], np_geo_coordi[:,0], np_geo_coordi[:,1]):
    m = folium.CircleMarker([la,ln], radius=10, popup=clean,
                            color='#3186cc',fill_color='#3186cc')
    m.add_to(jeju_map)
jeju_map

In [13]:
import folium
import json
import warnings
import pandas as pd

warnings.simplefilter(action='ignore', category=FutureWarning)

In [25]:
pop = pd.read_csv('3.csv')
pop.head()

,군집,CODE
0,1,3901054
1,1,3901065
2,1,3901066
3,3,3901012
4,3,3901052


In [ ]:
jeju_map = folium.Map(location=[33.363600, 126.528222], zoom_start = 10)
jeju_map.choropleth(geo_data=state_geo, data=state_data, 
               columns=['군집', 'CODE'],key_on = 'feature.properties.adm_cd',
               fill_color='PuRd', fill_opacity=0.01)

In [ ]:
state_geo = 'jeju.geojson'
state_unemployment = '4.csv'
state_data = pd.read_csv(state_unemployment)
state_data.columns = ['군집', 'CODE']
state_data['CODE'] = state_data.CODE.map(lambda x : str(x).zfill(7))
state_data.head(1)

In [15]:
geo_path = 'jeju.geojson'
geo_data = json.load(open(geo_path))
map =folium.Map(location=[33.363600, 126.528222], zoom_start=10)
map.choropleth(geo_data,
              data= pop['인구'],
              columns = [pop.index, pop['인구']],
              fill_color='YlGnBu')
map

In [23]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [24]:
client_id = "j2nijytx8x"
client_pw = "D12z0TZrLISsTlZMNaDejzgrvYZnbn4rhdViIoKz"

In [ ]:
api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

data = pd.read_excel('cleanhouse.xlsx', usecols='A, B, C',names=['읍면동', '도움센터명칭','주소'])

# 위도경도 찾기
geo_coordi = []
for add in data['주소']:
    add_urlenc = parse.quote(add)
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID',client_id)
    request.add_header('X-NCP-APIGW-API-KEY',client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'addresses' in response_body:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
            else:
                print("'result' not exist!")
                latitude = None
                longitude  = None
        else:
            print('Response error code : %d' %rescode)
            latitude = None
            longitude = None
            
    geo_coordi.append([latitude, longitude])

In [ ]:
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"읍면동":data["읍면동"].values,
                              "도움센터명칭":data["도움센터명칭"].values,
                              "도로명":data['주소'].values,
                              "위도": np_geo_coordi[:,0],
                              "경도": np_geo_coordi[:,1]})

In [ ]:
writer = pd.ExcelWriter('clean_h.xlsx')
pd_geo_coordi.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [ ]:
import folium

for clean, la, ln in zip(data["도움센터명칭"], np_geo_coordi[:,0], np_geo_coordi[:,1]):
    m = folium.Marker([la,ln], popup=clean)
    m.add_to(jeju_map)
map

In [ ]:
for clean, la, ln in zip(data["도움센터명칭"], np_geo_coordi[:,0], np_geo_coordi[:,1]):
    m = folium.CircleMarker([la,ln], radius=10, popup=clean,
                            color='#3186cc',fill_color='#3186cc')
    m.add_to(map)
map